In [ ]:
!rm -Rf sample_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [ ]:
!pip install -q timm
!pip install -q transformers

In [ ]:
# %%capture
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-4pb7jkr5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-4pb7jkr5
  Resolved https://github.com/facebookresearch/detectron2.git to commit 3ff5dd1cff4417af07097064813c9f28d7461d3c
  Preparing metadata (setup.py) ... done
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
  Using cached fvcore-0.1.5.post20221221.t

In [ ]:
import shutil
import torch
from torch.utils.data import DataLoader,Dataset
import transformers
from transformers import CLIPModel, CLIPConfig
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import cv2 as cv
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import WeightedRandomSampler
from sklearn.metrics import  f1_score
from tqdm.autonotebook import tqdm
import json
from copy import deepcopy
from getpass import getpass
import urllib


from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures.image_list import ImageList
from detectron2.data import transforms as T
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers
from detectron2.structures.boxes import Boxes
from detectron2.layers import nms, ShapeSpec
# from detectron2.model_zoo.model_z oo import get_config_file, get_checkpoint_url
from detectron2 import model_zoo
from detectron2.config import get_cfg



from transformers import BertTokenizer, VisualBertForPreTraining, VisualBertModel

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


from tensorboard.plugins import projector

In [ ]:
len(os.listdir('train_images')), len(os.listdir('val_images'))

(7500, 1000)

In [ ]:
sentiment_dict = {
    'positive' : 2,
    'neutral' : 1,
    'negative' : 0
}

In [ ]:
train_df = pd.read_csv('train_data.csv') # PATH TO train.csv
train_df['overall_sentiment'] = train_df['overall_sentiment'].replace(to_replace=sentiment_dict)
train_df.head()

,name,url,ocr_text,humour,sarcastic,offensive,motivational,overall_sentiment,classification_based_on
0,3449.jpg,https://i.pinimg.com/originals/8f/f3/63/8ff363...,where do you see yourself in 30 years memechete,funny,not_sarcastic,not_offensive,motivational,2,image_and_text
1,imgur_6_812.jpg,https://i.imgur.com/QeqzcNe_d.webp?maxwidth=52...,we re tate ratic party the future nevada stati...,funny,extremely_sarcastic,not_offensive,not_motivational,1,image
2,gg_1_313.jpg,https://img-9gag-fun.9cache.com/photo/ayM6ddp_...,ll vodacom 0949 32 meme 9h think 9gag think ...,not_funny,little_sarcastic,slight,not_motivational,0,image_and_text
3,cheezeburger3_253.jpeg,https://i.chzbgr.com/full/9600059648/hDE0E28FD...,she likes to collect rocks,very_funny,not_sarcastic,not_offensive,not_motivational,1,image_and_text
4,4137.jpg,https://i.pinimg.com/originals/b3/ec/f4/b3ecf4...,when your friends think you are not natural be...,funny,not_sarcastic,not_offensive,not_motivational,1,image_and_text


In [ ]:
test_df = pd.read_csv('val_data.csv') # PATH TO val.csv
test_df['overall_sentiment'] = test_df['overall_sentiment'].replace(to_replace=sentiment_dict)
test_df.head()

,name,url,ocr_text,humour,sarcastic,offensive,motivational,overall_sentiment,classification_based_on
0,2680.jpg,https://i.pinimg.com/originals/9e/69/4e/9e694e...,me after learning a word in french,funny,not_sarcastic,not_offensive,not_motivational,1,image_and_text
1,6179.jpg,https://preview.redd.it/u5a0ihi6tq861.jpg?widt...,goodbye flash my kiddad what is adobe flash me...,funny,little_sarcastic,slight,not_motivational,1,image_and_text
2,imgur_8_0978.jpg,https://pics.me.me/thumb_i-like-baguette-by-au...,no one how the world sees the french,funny,not_sarcastic,slight,not_motivational,0,image_and_text
3,imgur_8_1573.jpg,https://pics.me.me/thumb_she-shouldn%E2%80%99t...,oft whoeone a human heart costs 557000 but you...,funny,not_sarcastic,not_offensive,not_motivational,0,image_and_text
4,imgur5_232.jpg,https://i.imgur.com/3XxEmAD_d.jpg?maxwidth=520...,me joking about my mental health people who ca...,hilarious,extremely_sarcastic,not_offensive,not_motivational,1,image_and_text


In [ ]:
train_df.dropna(inplace=True), test_df.dropna(inplace=True)

(None, None)

In [ ]:
train_df.shape, test_df.shape

((7500, 9), (1000, 9))

In [ ]:
len(train_df.name.unique()), len(test_df.name.unique())

(7500, 1000)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
'''
  Dataset class
'''
class MemotionData(Dataset):


    def __init__(self,root_dir,img_names,ocr_text,sentiments,tokeniser, transforms=None):

        self.img_names = img_names
        self.ocr_text = ocr_text
        self.sentiments = sentiments
        self.root_dir = root_dir
        self.transforms = transforms
        self.tokeniser = tokeniser

    def __len__(self):
        return len(self.img_names)


    def __getitem__(self,idx):

        # print(self.img_names[idx])

        img = cv.imread(os.path.join(self.root_dir,self.img_names[idx]))
        img = cv.resize(img, (224,224))
        # img = cv.cvtColor(img,cv.COLOR_BGR2RGB)

        if self.transforms:
            img = self.transforms(img)




        tokens = self.tokeniser(self.ocr_text[idx], padding='max_length', max_length=512 )

        input_ids = torch.tensor(tokens["input_ids"])
        attention_mask = torch.tensor(tokens["attention_mask"])
        token_type_ids = torch.tensor(tokens["token_type_ids"])

        return {
            'img': img,
            'ocr_text': self.ocr_text[idx],
            'sentiment': torch.tensor(self.sentiments[idx],dtype=torch.long),
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }



def create_dataset(df,tokeniser, folder):
    ds = MemotionData(
        root_dir = folder,
        img_names = df['name'].to_list(),
        ocr_text= df['ocr_text'].to_list(),
        sentiments = df['overall_sentiment'].to_list(),
        tokeniser= tokeniser
    )
    return ds

# Sentiment Analysis

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Create dataset
train_dataset = create_dataset(train_df, tokenizer,  'train_images')
test_dataset = create_dataset(test_df, tokenizer, 'test_images')
# val_dataset = create_dataset(df_val,tokeniser, 'train_images')

In [ ]:
# for x in train_dataset:
#   print(x['input_ids'].shape, x['token_type_ids'].shape)

In [ ]:
'''
  Return Weights for training
'''

def ret_sample_weights(df_new,train_dataset):

  class_counts = list(np.unique(df_new['overall_sentiment'],return_counts=True)[1])
  class_weights = [sum(class_counts)/c for c in class_counts]

  sample_weights = [0]*len(train_dataset)

  for idx,x in enumerate(train_dataset):
    class_weight = class_weights[x['sentiment']]
    sample_weights[idx] = class_weight

  return sample_weights


In [ ]:
sample_weights = ret_sample_weights(train_df,train_dataset)

In [ ]:
len(sample_weights)

7500

In [ ]:
'''
  Loaders for training
'''
BATCH_SIZE = 16

train_sampler = WeightedRandomSampler(sample_weights,num_samples=len(sample_weights),replacement=True)
train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,sampler=train_sampler, drop_last=True)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE)

In [ ]:
# for i in train_loader:
#     # print(i.keys())
#     print((i['sentiment']).shape)
#     # break

In [ ]:
class FasterRCNNEmbeddings(torch.nn.Module):

    def __init__(self, cfg_path, device, num_classes=3 ):
        super(FasterRCNNEmbeddings, self).__init__()
        self.cfg_path = cfg_path
        self.device = device

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.bert_model = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre').to(device) # this checkpoint has 1024 dimensional visual embeddings projection
        self.num_classes = num_classes

        self.fc1 = nn.Linear(768, 256).to(device)
        self.relu = nn.ReLU().to(device)
        self.batchnorm = nn.BatchNorm1d(256).to(device)
        self.fc2 = nn.Linear(256, self.num_classes).to(device)



        # Load Config and Model Weights

        self.cfg = self.load_config_and_model_weights(self.cfg_path)

        # Load Object detection model

        self.od_model = self.get_model(self.cfg)

        for params in self.od_model.parameters():
          params.requires_grad = False

        for params in self.bert_model.parameters():
          params.requires_grad = False


    def load_config_and_model_weights(self,cfg_path):
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file(cfg_path))

        # ROI HEADS SCORE THRESHOLD
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

        # Comment the next line if you're using 'cuda'
        # cfg['MODEL']['DEVICE']='cpu'

        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(cfg_path)

        return cfg

    def get_model(self,cfg):
        # build model
        model = build_model(cfg)

        # load weights
        checkpointer = DetectionCheckpointer(model)
        checkpointer.load(cfg.MODEL.WEIGHTS)

        # eval mode
        model.eval()
        return model

    def prepare_image_inputs(self, cfg, img_list, model):

        # Resizing the image according to the configuration
        transform_gen = T.ResizeShortestEdge(
                    [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
                )
        img_list = [transform_gen.get_transform(img).apply_image(img) for img in img_list]

        # Convert to C,H,W format
        convert_to_tensor = lambda x: torch.Tensor(x.astype("float32").transpose(2, 0, 1))

        batched_inputs = [{"image":convert_to_tensor(img), "height": img.shape[0], "width": img.shape[1]} for img in img_list]

        # Normalizing the image
        num_channels = len(cfg.MODEL.PIXEL_MEAN)
        pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).view(num_channels, 1, 1)
        pixel_std = torch.Tensor(cfg.MODEL.PIXEL_STD).view(num_channels, 1, 1)
        normalizer = lambda x: (x - pixel_mean) / pixel_std
        images = [normalizer(x["image"]) for x in batched_inputs]

        # Convert to ImageList
        images =  ImageList.from_tensors(images,model.backbone.size_divisibility)

        return images, batched_inputs


    def get_features(self,model, images):

        features = model.backbone(images.tensor.to(device))
        return features


    def get_proposals(self, model, images, features):

        proposals, _ = model.proposal_generator(images, features)
        return proposals

    def get_box_features(self, model, features, proposals):

        # Get fc2 outputs in ROI Heads

        features_list = [features[f] for f in ['p2', 'p3', 'p4', 'p5']]
        box_features = model.roi_heads.box_pooler(features_list, [x.proposal_boxes for x in proposals])
        box_features = model.roi_heads.box_head.flatten(box_features)
        box_features = model.roi_heads.box_head.fc1(box_features)
        box_features = model.roi_heads.box_head.fc_relu1(box_features)
        box_features = model.roi_heads.box_head.fc2(box_features)

        box_features = box_features.reshape(len(proposals), 1000, 1024) # depends on your config and batch size
        return box_features, features_list

    def get_prediction_logits(self, model, features_list, proposals):

        cls_features = model.roi_heads.box_pooler(features_list, [x.proposal_boxes for x in proposals])
        cls_features = model.roi_heads.box_head(cls_features)
        pred_class_logits, pred_proposal_deltas = model.roi_heads.box_predictor(cls_features)

        return pred_class_logits, pred_proposal_deltas


    def get_box_scores(self, cfg, pred_class_logits, pred_proposal_deltas, proposals):

        box2box_transform = Box2BoxTransform(weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS)
        smooth_l1_beta = cfg.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA

        outputs = FastRCNNOutputLayers(
            input_shape = ShapeSpec(*proposals[0].image_size),  # We are not using it but this parameter is necessary to initialise FastRCNNOutputLayers
            box2box_transform=box2box_transform,
            smooth_l1_beta=smooth_l1_beta,
            num_classes = 80 # Doesn't matter
        )

        boxes = outputs.predict_boxes((pred_class_logits,pred_proposal_deltas),proposals)
        scores = outputs.predict_probs((pred_class_logits,pred_proposal_deltas),proposals)

        return boxes, scores


    def get_output_boxes(self, boxes, batched_inputs, image_size):

        proposal_boxes = boxes.reshape(-1, 4).clone()
        scale_x, scale_y = (batched_inputs["width"] / image_size[1], batched_inputs["height"] / image_size[0])
        output_boxes = Boxes(proposal_boxes)

        output_boxes.scale(scale_x, scale_y)
        output_boxes.clip(image_size)

        return output_boxes


    def select_boxes(self, cfg, output_boxes, scores):

        test_score_thresh = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST
        test_nms_thresh = cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST
        cls_prob = scores.cpu().detach()
        cls_boxes = output_boxes.tensor.cpu().detach().reshape(1000,80,4)
        max_conf = torch.zeros((cls_boxes.shape[0]))
        for cls_ind in range(0, cls_prob.shape[1]-1):
            cls_scores = cls_prob[:, cls_ind+1]
            det_boxes = cls_boxes[:,cls_ind,:]
            keep = np.array(nms(det_boxes, cls_scores, test_nms_thresh))
            max_conf[keep] = torch.where(cls_scores[keep] > max_conf[keep], cls_scores[keep], max_conf[keep])
        keep_boxes = torch.where(max_conf >= test_score_thresh)[0]

        return keep_boxes, max_conf



    def filter_boxes(self, keep_boxes, max_conf, min_boxes, max_boxes):

        if len(keep_boxes) < min_boxes:
            keep_boxes = np.argsort(max_conf).numpy()[::-1][:min_boxes]
        elif len(keep_boxes) > max_boxes:
            keep_boxes = np.argsort(max_conf).numpy()[::-1][:max_boxes]
        return keep_boxes

    def get_visual_embeds(self, box_features, keep_boxes):

        return box_features[keep_boxes.copy()]




    def forward(self, img_list, ocr_text, input_ids, attention_mask, token_type_ids ):

        # Convert Image List to Model Input

        images, batched_inputs = self.prepare_image_inputs(self.cfg, img_list, self.od_model)

        # Get FPN Features

        features = self.get_features(self.od_model, images)

        # Get Region Proposals

        proposals = self.get_proposals(self.od_model, images, features)

        # Get Box features from ROI Heads

        box_features, features_list = self.get_box_features(self.od_model, features, proposals)

        # Get prediction logits

        pred_class_logits, pred_proposal_deltas = self.get_prediction_logits(self.od_model, features_list, proposals)

        # Get Box with Scores

        boxes, scores = self.get_box_scores(self.cfg, pred_class_logits, pred_proposal_deltas, proposals)

        #  Rescaled box size ( detectron2 requires this )

        output_boxes = [self.get_output_boxes(boxes[i], batched_inputs[i], proposals[i].image_size) for i in range(len(proposals))]

        # Select Boxes using NMS

        temp = [self.select_boxes(self.cfg, output_boxes[i], scores[i]) for i in range(len(scores))]
        keep_boxes, max_conf = [],[]
        for keep_box, mx_conf in temp:
            keep_boxes.append(keep_box)
            max_conf.append(mx_conf)

        # Limit number of boxes

        MIN_BOXES=10
        MAX_BOXES=100
        keep_boxes = [self.filter_boxes(keep_box, mx_conf, MIN_BOXES, MAX_BOXES) for keep_box, mx_conf in zip(keep_boxes, max_conf)]

        # Get Embeddings

        visual_embeds = [self.get_visual_embeds(box_feature, keep_box) for box_feature, keep_box in zip(box_features, keep_boxes)]

        visual_embeds = torch.stack(visual_embeds)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long).to(self.device)
        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long).to(self.device)


        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, visual_embeds=visual_embeds, visual_attention_mask=visual_attention_mask, visual_token_type_ids=visual_token_type_ids, output_hidden_states=True)

        out =  self.fc1(outputs.pooler_output)
        out = self.batchnorm(out)
        out = self.relu(out)

        out = self.fc2(out)

        return out



In [ ]:
cfg_path = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

vis_model = FasterRCNNEmbeddings(cfg_path, device, 3)


Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425M [00:00<?, ?B/s]

Some weights of the model checkpoint at uclanlp/visualbert-nlvr2-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
model_final_f6e8b1.pkl: 243MB [00:07, 34.1MB/s]                           


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vis_model.parameters(), lr=3e-5, betas=(0.5,0.99))

In [ ]:
final_score = 0

In [ ]:
for epoch in range(30):

    print('Epoch ',epoch+1)

    running_loss = 0.0

    correct = 0
    total = 0
    training_loss = []
    y_true = []
    y_predicted = []

    for i, x in enumerate(train_loader):
        # print(x['img'].shape)

        optimizer.zero_grad()
        outputs = vis_model(x['img'].numpy(),x['ocr_text'], x['input_ids'].to(device),x['attention_mask'].to(device),x['token_type_ids'].to(device))

        _, predicted = torch.max(outputs.data, 1)
        total += x['sentiment'].to(device).size(0)
        y_true.append(x['sentiment'].to(device))
        y_predicted.append(predicted)
        correct += (predicted.cpu() == x['sentiment']).sum().item()

        loss = criterion(outputs, x['sentiment'].to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        training_loss.append(loss.item())
        if (i+1) % 50 == 0:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    y_true = torch.cat(y_true,dim=0)
    y_predicted = torch.cat(y_predicted,dim=0)

    train_f1_s = f1_score(y_true.cpu().numpy(),y_predicted.cpu().numpy(),average='macro')

    print(f'Train Acc : {100*correct/total}')
    print(f'Train F1 : {train_f1_s}')
    print(f'Val Acc : ',end=' ')

    correct = 0
    total = 0
    y_true = []
    y_predicted = []

    with torch.no_grad():
        for x in test_loader:

            labels = x['sentiment'].to(device)

            outputs = vis_model(x['img'].numpy(),x['ocr_text'], x['input_ids'].to(device),x['attention_mask'].to(device),x['token_type_ids'].to(device))

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            y_true.append(x['sentiment'].to(device))
            y_predicted.append(predicted)
            correct += (predicted == labels).sum().item()

    y_true = torch.cat(y_true,dim=0)
    y_predicted = torch.cat(y_predicted,dim=0)
    val_f1_s = f1_score(y_true.cpu().numpy(),y_predicted.cpu().numpy(),average='weighted')
    print(100 * correct / total)
    print('Val Macro F1',val_f1_s)
    # print(y_true)
    # print(y_predicted)
    # if f1_s > final_score:
    #     final_score = f1_s
    #     torch.save(senti_model.state_dict(),'/content/drive/MyDrive/Wipro/Final/Task A/senti_models/memotion_sentences_clip_memotion_taska.pt')
    # writer.add_scalar('Val Acc',100*correct/total,epoch)
    # writer.add_scalar('Val Macro F1',f1_s,epoch)

print('Finished Training')

Epoch  1


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[1,    50] loss: 1.12769
